In [96]:
import os
os.chdir("c:\\Users\\prabhav.gupta\\OneDrive - In2IT Technologies Pvt Ltd\\Documents\\AIOps\\MLOps\\MLOps")

In [97]:
pwd


'c:\\Users\\prabhav.gupta\\OneDrive - In2IT Technologies Pvt Ltd\\Documents\\AIOps\\MLOps\\MLOps'

In [98]:
import dagshub
dagshub.init(repo_owner='prabhav131', repo_name='MLOps', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Initialized MLflow to track repo "prabhav131/MLOps"

[2024-05-29 17:23:43,713: INFO: helpers: Initialized MLflow to track repo "prabhav131/MLOps"]


Repository prabhav131/MLOps initialized!

[2024-05-29 17:23:43,716: INFO: helpers: Repository prabhav131/MLOps initialized!]


In [100]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/prabhav131/MLOps.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "prabhav131"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "7a57c6a16d8bdede4119a82de1fd4e2fb08645e6"

In [101]:
import tensorflow as tf

In [103]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [104]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [105]:
from mlproject.constants import *
from mlproject.utils.common import read_yaml, create_directories, save_json

In [106]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/prabhav131/MLOps.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [107]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [120]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

                

    def log_into_mlflow(self):
        try:
            # Set tracking URIs
            mlflow.set_tracking_uri(self.config.mlflow_uri)
            
            # Set the experiment
            experiment_name = "your_experiment_name"
            mlflow.set_experiment(experiment_name)
            
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            
            # Start a new MLflow run
            with mlflow.start_run():
                # Log parameters and metrics
                if isinstance(self.config.all_params, dict):
                    mlflow.log_params(self.config.all_params)
                else:
                    raise ValueError("all_params should be a dictionary")
                    
                if len(self.score) == 2:
                    mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
                else:
                    raise ValueError("score should contain exactly 2 elements: loss and accuracy")
                
                

        except mlflow.exceptions.MlflowException as e:
            print(f"MLflowException: {e}")
            raise
        except Exception as e:
            print(f"General Exception: {e}")
            raise
                

In [121]:
config = ConfigurationManager()


[2024-05-29 18:07:41,857: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-29 18:07:41,858: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-29 18:07:41,860: INFO: common: created directory at: artifacts]


In [122]:
eval_config = config.get_evaluation_config()


In [123]:
evaluation = Evaluation(eval_config)


In [124]:
evaluation.evaluation()


Found 139 images belonging to 2 classes.


c:\Users\prabhav.gupta\OneDrive - In2IT Technologies Pvt Ltd\Documents\AIOps\MLOps\MLOps\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.2985 - loss: 10.6826
[2024-05-29 18:08:00,408: INFO: common: json file saved at: scores.json]


In [126]:
try:
    evaluation.log_into_mlflow()
except Exception as e:
    raise e    

KeyboardInterrupt: 